## Imports

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import urllib2
import textract
import os

## Global Values

Setting the folders to read the data from

In [2]:
# parent folder of the documents
parent_folder = ""
# folder to save pre-processed minutes in
pre_processed_folder = "pre_processed/"

## Defining different methods

Scrapping the urls of the minutes from the main page

In [3]:
def retrieve_urls(year = 2020):
    # Get urls of ecb Monetary policy accounts
    page_urls = []
    for i in range(2015, year):
        ecb_page = requests.get("https://www.ecb.europa.eu/press/accounts/" + str(i) + "/html/index.en.html")
        ecb_soup = BeautifulSoup(ecb_page.content, 'html.parser')
        for block in ecb_soup.select(".doc-title"):
            for url in block.select("a"):
                page_urls.append("https://www.ecb.europa.eu" + str(url["href"]))
    return page_urls

Simple methode to save a text in a given folder including creating folder and file (if not already existing)

In [36]:
def save_content(content, folder, filename, date):
    if os.path.exists(parent_folder + pre_processed_folder + date) == False:
        os.makedirs(parent_folder + pre_processed_folder + date)     
    
    with open(folder + "/" + filename + ".txt", "w") as file:
        try:
            file.write(content)
        except UnicodeEncodeError:
            file.write(str(content.encode("utf-8")))
        except TypeError:
            file.write(str(content))

Method scraps the sourcecode of the given url and extracts the content in it. Since the ECB is using a very clear structure for their website, it's easy to identify the article in it.

In [62]:
def filter_text(url, save = False):
    date_temp = "".join(re.findall('\d\d\d\d\d\d', url))
    date = "20" + date_temp
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    article = soup.find(class_ = 'ecb-pressContent')
    counter = 1
    for block in article.select("p"):
        text = block.get_text()
        if len(text) > 200 and save:
            save_content(content = text, folder = parent_folder + pre_processed_folder + date,filename = date + "_" + str(counter), date = date)
            counter = counter + 1

Retrieving all urls until the specified parameter and download the content of the websites.

In [63]:
urls = retrieve_urls(2019)
for url in urls:
    filter_text(url, save=True)